<a href="https://colab.research.google.com/github/alexlautw9527/for_notebook/blob/main/Fund2vec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
%%capture
!pip install node2vec
!pip install glove-python-binary

In [1]:
import argparse
import pprint

import pandas as pd 
import numpy as np


import networkx as nx
from node2vec import Node2Vec
import gensim
from gensim.models.word2vec import Word2Vec

from glove import Glove
from glove import Corpus

ModuleNotFoundError: ignored

## 整0.1資料

In [2]:
fund_code_df = pd.read_csv('/content/drive/MyDrive/05_OTHER_DATA/FUND_CODE_DATA.csv', encoding='CP950')
fund_code_df = fund_code_df[['產品代碼', '基金名稱', 'ISIN Code']]

In [3]:
fund_df = pd.read_csv('/content/drive/MyDrive/05_OTHER_DATA/fund_desc.csv', encoding='CP950')
fund_df['CNAME_short'] = fund_df['CNAME'].str.replace('\（[^（）]*\）', '', regex=True)


In [4]:
fund_df = fund_df[fund_df['RISK_LEVEL_ALL']!=0]
fund_df['基金類型'] = fund_df['基金類型'].fillna('NA')

In [5]:
fund_df = fund_df.astype(str)

In [6]:

index = ['FUND_CD']

cate_list = [
 'RISK_LEVEL_ALL',
 'FUNDCRYID',
 'RISKLEVEL',
 '配息',
 '股債分類',
 '基金類型',
 '產品類型',
 '基金分類',
 'SITCA分類',
 #'SITCA細類',
 #'追蹤指標',
 '境外內',
 '前後收'            
]


weight_list = [
 '股票比',
 '債券比',
 '現金比',
 '其他比',
 '成熟股',
 '成熟股_美國',
 '成熟股_北美',
 '成熟股_歐洲',
 '成熟股_英',
 '成熟股_日本',
 '成熟股_亞太',
 '新興股',
 '新興股_拉美',
 '新興股_新歐',
 '新興股_非洲',
 '新興股_中東',
 '新興股_新亞',
 '原物料',
 '非必消',
 '金融',
 '房地產',
 '電信',
 '能源',
 '工業',
 '科技',
 '必需消',
 '健護',
 '公用事業',
 '成熟債',
 '成熟債_美國',
 '成熟債_北美',
 '成熟債_歐洲',
 '成熟債_英',
 '成熟債_日本',
 '成熟債_亞太',
 '新興債',
 '新興債_拉美',
 '新興債_新歐',
 '新興債_非洲',
 '新興債_中東',
 '新興債_新亞',
 'AAA',
 'AA',
 'A',
 'BBB',
 'BB',
 'B',
 'B以下',
 '無信評'
]

all_list = index+cate_list+weight_list

In [7]:
fund_df = fund_df[fund_df['風屬']!='nan']
fund_org_df = fund_df.copy()

fund_df = fund_df[all_list]
fund_df = fund_df.replace('nan', np.nan)

In [8]:
fund_org_df.shape

(1891, 250)

In [10]:
fund_df[weight_list] = fund_df[weight_list].astype(float).fillna(0)
fund_df[weight_list] = fund_df[weight_list]/100

## Node2vec

In [11]:
fund_df['產品類型'].unique()

array(['債券型', '股票型', '平衡型', '貨幣型'], dtype=object)

In [12]:
(fund_df['配息'].unique()).shape

(3,)

In [25]:
fund_df[all_list].to_csv('haha.csv',index=False, encoding='CP950')

In [14]:
## 製作graph

fund_edge = pd.get_dummies(
  fund_df[all_list], 
  columns=cate_list, prefix='', prefix_sep=''
)

In [15]:
fund_edge = fund_edge.set_index('FUND_CD').stack()
fund_edge = fund_edge[fund_edge>0]

In [16]:
fund_edge = fund_edge.reset_index().values

In [17]:
fund_edge

array([['AOB4', '債券比', 0.9891],
       ['AOB4', '現金比', 0.01],
       ['AOB4', '其他比', 0.0009],
       ...,
       ['WAB1', '股票型', 1.0],
       ['WAB1', '境外', 1.0],
       ['WAB1', '前收', 1.0]], dtype=object)

In [18]:
G = nx.Graph()

for x in list(fund_edge):
  G.add_edge(x[0], x[1], weight=x[2])

In [19]:
plot_G = nx.Graph()

for x in list(fund_edge[0:100]):
  plot_G.add_edge(x[0], x[1], weight=x[2])

In [20]:

!pip install pyvis

In [21]:
%%capture
# pyvis part

from pyvis.network import Network
nt = Network("500px", "1000px", notebook=False)
nt.from_nx(plot_G)


In [22]:

from IPython.core.display import display, HTML

nt.show('network.html')
display(HTML('network.html'))


In [ ]:
%%time
'''
:param graph: Input graph
:param dimensions: Embedding dimensions (default: 128)
:param walk_length: Number of nodes in each walk (default: 80)
:param num_walks: Number of walks per node (default: 10)
:param p: Return hyper parameter (default: 1)
:param q: Inout parameter (default: 1)
:param weight_key: On weighted graphs, this is the key for the weight attribute (default: 'weight')
:param workers: Number of workers for parallel execution (default: 1)
:param sampling_strategy: Node specific sampling strategies, supports setting node specific 'q', 'p', 'num_walks' and 'walk_length'.
:param seed: Seed for the random number generator.
'''

# Precompute probabilities and generate walks - **ON WINDOWS ONLY WORKS WITH workers=1**

node2vec = Node2Vec(G, dimensions=64, walk_length=50, num_walks=20, workers=1, seed=9527)  # Use temp_folder for big graphs

Computing transition probabilities:   0%|          | 0/2044 [00:00<?, ?it/s]

Generating walks (CPU: 1): 100%|██████████| 20/20 [02:22<00:00,  7.14s/it]

CPU times: user 5min 15s, sys: 2.4 s, total: 5min 17s
Wall time: 5min 15s


In [ ]:
node2vec.walks[0]

['FDU1',
 '4',
 'AOA7',
 '非必消',
 'MSG9',
 '資訊科技股票',
 'MSH1',
 'STOCK',
 'ALM7',
 '科技',
 'FRP5',
 '股票型',
 'JAA8',
 'STOCK',
 'SCO1',
 '境外',
 'JFL1',
 '境外',
 'JFU8',
 '股票比',
 '3914',
 'STOCK',
 'FDM1',
 '成熟股',
 'JAA4',
 '成熟股',
 '3635',
 '4',
 '8111',
 '4',
 'FDS5',
 'STOCK',
 'FDK7',
 '成熟股_歐洲',
 'SCR2',
 '歐洲股票',
 'AEA4',
 '股票比',
 '6202',
 '後收',
 'FFA6',
 '北美股票',
 'UBA5',
 '工業',
 '1336',
 '新興股',
 'KBA1',
 '成熟股_北美',
 '7799',
 '環球股票']

## 語料庫訓練

In [ ]:
node_corpus = node2vec.walks

In [ ]:
%%time
n2v_model = Word2Vec(node_corpus, size=256, iter=5, window=8, sg=1, negative=10, min_count=1, workers=1)

CPU times: user 2min 18s, sys: 193 ms, total: 2min 18s
Wall time: 2min 17s


In [ ]:
n2v_model.save("fund_node2vec.model")

In [ ]:
fund_df['基金類型'].unique().shape

(47,)

In [ ]:
n2v_mkt_emb = {x:n2v_model.wv[x] for x in fund_df['基金類型'].unique()}

In [ ]:
np.savetxt("n2v_mkt_emb.tsv", n2v_model.wv[fund_df['基金類型'].unique()], delimiter="\t")
pd.DataFrame({'mkt':fund_df['基金類型'].unique()}).to_csv('n2v_mkt_emb_meta.tsv', sep = '\t',index=False, header=False)

In [ ]:
fund_org_df

In [ ]:
np.savetxt("n2v_mkt_emb.tsv", n2v_model.wv[fund_df['基金類型'].unique()], delimiter="\t")
pd.DataFrame({'mkt':fund_df['基金類型'].unique()}).to_csv('n2v_mkt_emb_meta.tsv', sep = '\t',index=False, header=False)

In [ ]:
np.savetxt("n2v_fund_emb.tsv", n2v_model.wv[fund_df['FUND_CD']], delimiter="\t")
fund_org_df[['CNAME_short','RISK_LEVEL_ALL','FUNDCRYID','產品類型','基金類型','基金分類',]].to_csv('n2v_fund_emb_meta.tsv', sep = '\t',index=False, header=False)

In [ ]:
!cp n2v_mkt_emb.tsv /content/drive/MyDrive/FUND2VEC_V2
!cp n2v_mkt_emb_meta.tsv /content/drive/MyDrive/FUND2VEC_V2

!cp n2v_fund_emb.tsv /content/drive/MyDrive/FUND2VEC_V2
!cp n2v_fund_emb_meta.tsv /content/drive/MyDrive/FUND2VEC_V2

## GLOVE

In [ ]:

#准备数据集
sentense = [['你','是','谁'],['我','是','台灣人']]
corpus_model = Corpus()
corpus_model.fit(sentense, window=10)


#corpus_model.save('corpus.model')
print('Dict size: %s' % len(corpus_model.dictionary))
print('Collocations: %s' % corpus_model.matrix.nnz)

#训练
glove = Glove(no_components=100, learning_rate=0.05)
glove.fit(corpus_model.matrix, epochs=10,
          no_threads=1, verbose=True)
glove.add_dictionary(corpus_model.dictionary)

#模型保存
glove.save('glove.model')
glove = Glove.load('glove.model')
#语料保存
corpus_model.save('corpus.model')
corpus_model = Corpus.load('corpus.model')

#求相似词
glove.most_similar('我', number=10)

ModuleNotFoundError: ignored

In [ ]:
def most_similar(w2v_model, words, topn=20):
    similar_df = pd.DataFrame()
    for word in words:
        try:
            similar_words = pd.DataFrame(w2v_model.wv.most_similar(word, topn=topn), columns=[word, 'cos'])
            similar_df = pd.concat([similar_df, similar_words], axis=1)
        except:
            print(word, "not found in Word2Vec model!")
    return similar_df

In [ ]:
from google.colab import files
files.download('/content/fund_embedd_metadata.tsv') 
files.download('/content/fund_embedd.tsv') 

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## Pretrained W2V combining

In [ ]:
fund_corpus = fund_df.values.tolist()

[['聯博新興市場債券基金ＥＡ（美元）（穩定月配息）（本基金進行配息前未先扣除應負擔之相關費用）（本基金有相當比重投資於非投資等級之高風險債券且配息來源可能為本金）',
  'USD',
  '3',
  '01_配息',
  'BOND',
  '新興市場債'],
 ['聯博新興市場債券基金ＥＡ（澳幣避險）（穩定月配息）（本基金進行配息前未先扣除應負擔之相關費用）（本基金有相當比重投資於非投資等級之高風險債券且配息來源可能為本金）',
  'AUD',
  '3',
  '01_配息',
  'BOND',
  '新興市場債'],
 ['聯博－新興市場債券基金ＥＡ（南非幣避險）（穩定月配息）（本基金進行配息前未先扣除應負擔之相關費用）（本基金有相當比重投資於非投資等級之高風險債券且配息來源可能為本金）',
  'ZAR',
  '4',
  '01_配息',
  'BOND',
  '新興市場債'],
 ['聯博全球高收益債券基金ＥＡ（美元）（穩定月配息）（本基金進行配息前未先扣除應負擔之相關費用）（本基金主要係投資於非投資等級之高風險債券且配息來源可能為本金）',
  'USD',
  '3',
  '01_配息',
  'BOND',
  '高收益債'],
 ['聯博－國際醫療基金ＥＤ（美元）（月配息）（基金之配息來源可能為本金）', 'USD', '4', '01_配息', 'STOCK', '健護'],
 ['聯博美國收益基金ＥＡ（美元）（穩定月配）（本基金進行配息前未先扣除應負擔之相關費用）（本基金有相當比重投資於非投資等級之高風險債券且配息來源可能為本金）',
  'USD',
  '3',
  '01_配息',
  'BOND',
  '全球債'],
 ['聯博亞洲股票基金ＥＤ（美元）（月配息）（本基金進行配息前未先扣除應負擔之相關費用）（基金之配息來源可能為本金）',
  'USD',
  '4',
  '01_配息',
  'STOCK',
  '亞太股市'],
 ['聯博美國成長基金ＥＤ（美元）（月配息）（本基金進行配息前未先扣除應負擔之相關費用）（基金之配息來源可能為本金）',
  'USD',
  '4',
  '01_配息',
  'STOCK',
  '美國股市'],
 ['聯博新興市場成長基金ＥＤ（美元）（月配息）（本基金進行配息前未

In [ ]:
! wget https://www.rahimikia.com/FinText/FinText_FastText_Skip-gram.zip

--2021-10-26 15:11:10--  https://www.rahimikia.com/FinText/FinText_FastText_Skip-gram.zip
Resolving www.rahimikia.com (www.rahimikia.com)... 69.163.228.90
Connecting to www.rahimikia.com (www.rahimikia.com)|69.163.228.90|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 11656495869 (11G) [application/zip]
Saving to: ‘FinText_FastText_Skip-gram.zip’

FinText_FastText_Sk 100%[===================>]  10.86G  13.1MB/s    in 8m 13s  

2021-10-26 15:19:24 (22.5 MB/s) - ‘FinText_FastText_Skip-gram.zip’ saved [11656495869/11656495869]



In [ ]:
!unzip /content/FinText_FastText_Skip-gram.zip

Archive:  /content/FinText_FastText_Skip-gram.zip
   creating: FinText_FastText_Skip-gram/
  inflating: FinText_FastText_Skip-gram/Word_Embedding_2000_2015  
  inflating: FinText_FastText_Skip-gram/README.txt  
  inflating: FinText_FastText_Skip-gram/Word_Embedding_2000_2015.trainables.vectors_ngrams_lockf.npy  
  inflating: FinText_FastText_Skip-gram/Word_Embedding_2000_2015.pkl  
  inflating: FinText_FastText_Skip-gram/Word_Embedding_2000_2015.wv.vectors_ngrams.npy  
  inflating: FinText_FastText_Skip-gram/Word_Embedding_2000_2015.wv.vectors.npy  
  inflating: FinText_FastText_Skip-gram/Word_Embedding_2000_2015.trainables.vectors_vocab_lockf.npy  
  inflating: FinText_FastText_Skip-gram/Word_Embedding_2000_2015.wv.vectors_vocab.npy  
  inflating: FinText_FastText_Skip-gram/Word_Embedding_2000_2015.trainables.syn1neg.npy  


In [ ]:
from gensim.models import Word2Vec
from gensim.models import FastText
import gensim.downloader as api 

In [ ]:
FinText_Word2Vec_skipgram = Word2Vec.load('/content/FinText_FastText_Skip-gram/Word_Embedding_2000_2015')

In [ ]:

prod=['fund',
 'etf',
 'stock',
 'preferred_stock',
 'auto_loan',
 'trust',
 'life_insurance',
 'insurance',
 'personal_loan',
 'mortgage',
 'loan',
 'treasury_bond',
 'foreign_exchange',
 'online_banking',
 'fintech',
 'blockchain',
 'bitcoin',
 'robo_advisor'
 ]

In [ ]:
product_embedding = FinText_Word2Vec_skipgram.wv[prod]

In [ ]:
product_embedding

array([[ 0.14604235,  0.1704098 ,  0.01929267, ..., -0.13392453,
        -0.3286863 ,  0.01605617],
       [ 0.40752319,  0.3141233 , -0.22489968, ..., -0.28437993,
         0.02246348,  0.17138132],
       [ 0.09119733,  0.13523722,  0.01482047, ..., -0.09712761,
        -0.23065072, -0.22710954],
       ...,
       [-0.00303637,  0.145203  , -0.41418603, ..., -0.11681274,
        -0.14459747, -0.02398938],
       [-0.07647095,  0.24916057, -0.12600106, ...,  0.3001527 ,
        -0.21294053, -0.04931631],
       [ 0.29539302, -0.00322265,  0.12641694, ...,  0.13217312,
        -0.4011273 , -0.07066225]], dtype=float32)

In [ ]:
product_embedding.shape

(18, 300)

In [ ]:
np.savetxt("product_embedd.tsv", product_embedding, delimiter="\t")
pd.DataFrame({'product':prod}).to_csv('prod_embedd_metadata.tsv', sep = '\t',index=False,header=False)

In [ ]:
pd.DataFrame({'product':prod}).to_csv('prod_embedd_metadata.tsv', sep = '\t',index=False,header=False)

In [ ]:
from google.colab import files
files.download('/content/prod_embedd_metadata.tsv') 
files.download('/content/product_embedd.tsv') 

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>